## Part 1

This one is basically one to ensure that we somehow get the lights that are needed on using the following format from the file:
- [..##..] = The default state of the lights
- List of tuples. Buttons indicating what lights to toggle when pressed.
- Set = the lights that are on after all the toggling which is the goal
- Buttons can be pressed multiple times.

We need to find the minimum number of button presses to get all the lights on. We are going to use Dijkstra's algorithm to find the shortest path to the goal state or BFS since all edges have the same weight but Dijkstra finds the minimal path which is important here.

In [3]:
# Code for Dijkstra evaluation
import heapq
from collections import defaultdict

def dijkstra(start, goal, buttons):
    queue = [(0, start)]
    visited = set()
    distances = defaultdict(lambda: float('inf'))
    distances[start] = 0

    while queue:
        current_distance, current_state = heapq.heappop(queue)

        if current_state in visited:
            continue
        visited.add(current_state)

        if current_state == goal:
            return current_distance

        for button in buttons:
            # Convert button to set for faster lookup
            button_set = set(button) if isinstance(button, (tuple, list)) else {button}
            next_state = tuple(
                not current_state[i] if i in button_set else current_state[i]
                for i in range(len(current_state))
            )
            distance = current_distance + 1

            if distance < distances[next_state]:
                distances[next_state] = distance
                heapq.heappush(queue, (distance, next_state))

    return -1  # Goal state not reachable

In [16]:
import ast

def transform_to_binary(state_str):
    new_list = []
    for char in list(state_str)[1:-1]:
        if char == '#':
            new_list.append(1)
        else:
            new_list.append(0)
    return new_list


# read input
combs = []
total = 0
with open("input.txt") as f:
    for line in f.readlines():
        line = line.strip().split(' ')
        default_state = tuple(transform_to_binary(line[0]))
        start_state = tuple([0] * len(default_state))
        line[-1] = line[-1].replace('{', '[', 1).replace('}', ']', 1)
        buttons_pressed_number = ast.literal_eval(line[-1])
        buttons = [ast.literal_eval(val) for val in line[1:-1]]
        combs.append((buttons, buttons_pressed_number))
        dijkstra_result = dijkstra(start_state, default_state, buttons)
        total += dijkstra_result

print("Final result:", total)

Final result: 441


## Part 2

The problem is different! We're not toggling switches anymore. Instead:
- We have counters that all start at 0
- Each button press INCREMENTS the counters at the specified positions by 1
- We need to reach the target counter values specified in {...}
- Find the minimum number of button presses needed

Example: [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
- Counters start at [0,0,0,0]
- Need to reach [3,5,4,7]
- Button (3) increments counter 3
- Button (1,3) increments counters 1 and 3
- etc.

This can be reformulated as a matrix equation and solved using integer linear programming:
- Matrix that all control a specific counter (where each button has a 1 if pressed it affects that counter, else 0)
- Vector of button presses (unknowns)
- Vector of target counter values (knowns)

Then using a linear programming solver to minimize the sum of button presses while satisfying the counter constraints.

In [19]:
from pulp import *

def solve_with_pulp(button_schematics, joltage_targets):
    """
    button_schematics: List of lists/tuples.
                       e.g. [(3,), (1, 3)] means Button 0 hits counter 3,
                       Button 1 hits counters 1 and 3.
    joltage_targets:   List of integers. e.g. [3, 5, 4, 7]
    """

    num_buttons = len(button_schematics)
    num_counters = len(joltage_targets)

    # 1. Initialize the Problem
    prob = pulp.LpProblem("Factory_Optimization", LpMinimize)
    # 2. Define Variables
    button_vars = pulp.LpVariable.dicts("btn",
                                        range(num_buttons),
                                        0,
                                        None,
                                        LpInteger)
    prob += pulp.lpSum([button_vars[i] for i in range(num_buttons)])
    for counter_idx in range(num_counters):
        # Find which buttons affect THIS specific counter
        affecting_buttons = []
        for btn_idx, effects in enumerate(button_schematics):
            if type(effects) is int:
                effects = (effects,)
            if counter_idx in effects:
                affecting_buttons.append(button_vars[btn_idx])

        prob += pulp.lpSum(affecting_buttons) == joltage_targets[counter_idx]

    # 5. Solve the problem
    status = prob.solve(pulp.PULP_CBC_CMD(msg=0))

    # 6. Check Result
    if LpStatus[status] == LpStatusOptimal:
        total_presses = int(pulp.value(prob.objective))
        return total_presses
    else:
        return 0

result = 0
for buttons, final_goal in combs:
    result += solve_with_pulp(buttons, final_goal)


In [20]:
result

18559